In [ ]:
"""
1.	Pre-requirments: Python 3.6 and python 2.7
2.	Install Tensorflow library using pip3"""

In [ ]:
#This Script is used by me to turn my train and test images important info into an xml file. xmin, ymin, xmax and ymax are 
#showing the coordination od bounding box around an object. as in my dataset the whole image was the object, I putted xmin and
#ymin equal to zero and xmax and ymax equal to the height and width of image.
#in case your dataset contains photos which objects are only a part of it, use labelimg application to draw bounding boxex,
#then an xml file is saved by labelimg automatically
#filename: script.py
from PIL import Image
import os
from xml.etree.ElementTree import parse, Element
folder = 'C:/Users/D071941/Documents/NLP/test'
save_path = 'C:/Users/D071941/Documents/NLP/testannotations'
for file in os.listdir(folder):
    filename = os.fsdecode(file)
    filepath = os.path.join(folder,filename)
    im = Image.open(filepath)
    width, height = im.size
    newfilename = filename.replace('.jpg','.xml')
    newfilepath = os.path.join(save_path, newfilename)
    tree = parse('C:/Users/D071941/Documents/NLP/xmlformat.xml')
    root = tree.getroot()
    tree.find('.//filename').text = filename
    tree.find('.//path').text = filepath
    tree.find('.//width').text = str(width)
    tree.find('.//height').text = str(height)
    tree.find('.//xmax').text = str(width)
    tree.find('.//ymax').text = str(height)
    if filename [6]=='0':
        tree.find('.//name').text = 'person'
    elif filename [6]=='1':
        tree.find('.//name').text = 'forest'
    elif filename [6]=='2':
        tree.find('.//name').text = 'highway'
    elif filename [6]=='4':
        tree.find('.//name').text = 'mountain'
    elif filename [6]=='5':
        tree.find('.//name').text = 'building'
    file_obj = open(newfilepath,"w") 
    tree.write(newfilepath, xml_declaration=True)

    


In [ ]:
#the "xmlformat.xml" used in previous code
<annotation>
	<folder>train</folder>
	<filename>image_0005.jpg</filename>
	<path>C:\Users\D071941\Documents\NLP\Faces_easy\image_0005.jpg</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>290</width>
		<height>330</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>building</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>0</xmin>
			<ymin>0</ymin>
			<xmax>284</xmax>
			<ymax>325</ymax>
		</bndbox>
	</object>
</annotation>


In [ ]:
#this file is called "xml2csvscript.py", it changes the xml file generated in previous step to CSV file.
#ofcourse you could only generate the csv from the start.
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text))
            xml_list.append(value)
    column_name = ['filename', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

def main():
    for i in ['C:/Users/D071941/Documents/NLP/testannotations']:
        folder = os.path.basename(os.path.normpath(i))
        xml_df = xml_to_csv(i)
        xml_df.to_csv(folder+'/test.csv', index=None)
        print('Successfully converted xml to csv.')
    
main()

In [ ]:
#turn .csv files into TF records
#you need path to images and path to csv files
"""
Usage:
  # From tensorflow/models/
  # Create train data:
  python generate_tfrecord.py --csv_input=data/train_labels.csv  --output_path=train.record
  # Create test data:
  python generate_tfrecord.py --csv_input=data/test_labels.csv  --output_path=test.record
"""
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

flags = tf.app.flags
flags.DEFINE_string('csv_input', 'data/test.csv', '')
flags.DEFINE_string('output_path', 'data/test.record', '')
FLAGS = flags.FLAGS


# TO-DO replace this with label map
def class_text_to_int(row_label):
    if row_label == 'person':
        return 1
    elif row_label == 'forest':
        return 2
    elif row_label == 'highway':
        return 3
    elif row_label == 'mountain':
	    return 4
    elif row_label == 'building':
	    return 5

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size
    print("file name:{}", group.filename)
    print("width:{}\theight:{}", width, height)
    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        print("xmin:{}", row['xmin'])
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):
    writer = tf.python_io.TFRecordWriter(FLAGS.output_path)
    path = os.path.join(os.getcwd(), 'test')
    examples = pd.read_csv(FLAGS.csv_input)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), FLAGS.output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))


if __name__ == '__main__':
    tf.app.run()

In [ ]:
"""
Now you have your dataset ready, you  could start training

3.	Find the directory of your python 3.6 and go to this path “Python36\Lib\site-packages\tensorflow”
4.	Retrieve the object detection API by running: 
5.	“git clone https://github.com/tensorflow/models”
6.	Install required libraries mentioned here: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md
7.	Now if you want to run on cloud go to this path in your computer:
“Python36\Lib\site-packages\tensorflow\models\research” 
And make the following changes in the required files:
•	Replace setup.py with this script:
"""
"""Setup script for object_detection."""

import logging
import subprocess
from setuptools import find_packages
from setuptools import setup
from setuptools.command.install import install

class CustomCommands(install):

	def RunCustomCommand(self, command_list):
		p = subprocess.Popen(
        command_list,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
		stdout_data, _ = p.communicate()
		logging.info('Log command output: %s', stdout_data)
		if p.returncode != 0:
			raise RuntimeError('Command %s failed: exit code: %s' %
                         (command_list, p.returncode))

	def run(self):
		self.RunCustomCommand(['apt-get', 'update'])
		self.RunCustomCommand(
          ['apt-get', 'install', '-y', 'python-tk'])
		install.run(self)

REQUIRED_PACKAGES = ['Pillow>=1.0', 'protobuf>=3.3.0', 'Matplotlib>=2.1']

setup(
    name='object_detection',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    include_package_data=True,
    packages=[p for p in find_packages() if p.startswith('object_detection')],
    description='Tensorflow Object Detection Library',
 cmdclass={
        'install': CustomCommands,
    }
)



"""
•	In object_detection directory, line 184 of evaluator.py, change
		tf.train.get_or_create_global_step()
to
		tf.contrib.framework.get_or_create_global_step()

•	In “samples\cloud” change the runtime version to 1.2, at the time I am writing this guide version 1.4 is released but it has some defects which leads to errors when you run Faster R-CNN and RFCN models.
•	In object_detection/utils/visualization_utils.py, line 24 (before import matplotlib.pyplot as plt) add:
		import matplotlib
		matplotlib.use('agg')

•	Finally, in line 103 of object_detection/builders/optimizer_builder.py, change
		 tf.train.get_or_create_global_step()
to
tf.contrib.framework.get_or_create_global_step()

"""

In [ ]:
# 8. From tensorflow/models/research/ run:
protoc object_detection/protos/*.proto --python_out=.
#9.You should add these paths to your system environment, So first replace the true path to your python36 in them and then
#Easily, add a new system variable called PYTHONPATH, add these paths to it
#go to the “path” in your system variables and add “%PYTHONPATH%” to it.
“C:\Program Files\Python36\Lib\site-packages\tensorflow\models”
“C:\Program Files\Python36\Lib\site-packages\tensorflow\models\research”
“C:\Program Files\Python36\Lib\site-packages\tensorflow\models\research\slim”
“C:\Program Files\Python36\Lib\site-packages\tensorflow\models\research\object_detection”

#10.	Test the installations: 
python object_detection/builders/model_builder_test.py

In [ ]:
"""
11.Set up the GCP project and bucket (name it something memorable) — and it should create ‘your first project’. 
You should do all the things mentioned in this link: https://cloud.google.com/solutions/creating-object-detection-application-tensorflow step by step. 
Then in your google cloud platform environment, from the storage in left hand nav bar, 
you could make a new bucket with a unique name. 
12.Enable Machine Learning (ML) Engine API’s — should be in the left hand nav bar of your project
13.nstall the Google Cloud SDK on your local machine, and do the authentication and choose your project
by the id of project you made in step 11.
"""

In [ ]:
"""
14. if you want to train locally, you should add a new folder to “C:\Program Files\Python36\Lib\site-packages\tensorflow\models\research\object_detection”
called mytraining for example. in this folder, you have: 1. TFrecord of train and test data 2. config file of the model
you want to train like faster_rcnn_inception_v2_coco.config --> obtain config files from this path:

C:\Program Files\Python36\Lib\site-packages\tensorflow\models\research\object_detection\samples\configs

3.you should have the initial checkpoints for the model to start. To obtain the checkpoints, open this file:

C:\Program Files\Python36\Lib\site-packages\tensorflow\models\research\object_detection\g3doc\detection_model_zoo.md

Download the zip file corresponding to your desired model from the links, you can file 3 check point file there. 4. you need a label map,
the label map, maps each class name to an int
add all of them to mytraining directory.
if you want to run on google cloud, add all these files to your bucket.
"""

In [ ]:
#sample config file mentioned in step 14
#rfcn_resnet101_coco.config
# R-FCN with Resnet-101 (v1),  configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.
#example of path on google cloud: "gs://object_detection_bucket_samin/data/train.record"

model {
  faster_rcnn {
    num_classes: 5
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_resnet101'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    second_stage_box_predictor {
      rfcn_box_predictor {
        conv_hyperparams {
          op: CONV
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            truncated_normal_initializer {
              stddev: 0.01
            }
          }
        }
        crop_height: 18
        crop_width: 18
        num_spatial_bins_height: 3
        num_spatial_bins_width: 3
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.0
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 300
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
  }
}

train_config: {
  batch_size: 1
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        manual_step_learning_rate {
          initial_learning_rate: 0.0003
          schedule {
            step: 0
            learning_rate: .0003
          }
          schedule {
            step: 900000
            learning_rate: .00003
          }
          schedule {
            step: 1200000
            learning_rate: .000003
          }
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  gradient_clipping_by_norm: 10.0
  fine_tune_checkpoint: "$PATH_TO_BE_CONFIGURED/model.ckpt"
  from_detection_checkpoint: true
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the pets dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  num_steps: 100000
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
}

train_input_reader: {
  tf_record_input_reader {
    input_path:  "$PATH_TO_BE_CONFIGURED/train.record"
  }
    
  label_map_path: "$PATH_TO_BE_CONFIGURED/label_map.pbtxt"
}

eval_config: {
  num_examples: 8000
  # Note: The below line limits the evaluation process to 10 evaluations.
  # Remove the below line to evaluate indefinitely.
  
}

eval_input_reader: {
  tf_record_input_reader {
    input_path:  "$PATH_TO_BE_CONFIGURED/test.record"
  }
  label_map_path: "$PATH_TO_BE_CONFIGURED/label_map.pbtxt"
  shuffle: false
  num_readers: 1
  num_epochs: 1
}


In [ ]:
#sample label map as mentioned in 14
#label_map.pbtxt
item {
  id: 1
  name: 'person'
}
item {
  id: 2
  name: 'forest'
}
item {
  id: 3
  name: 'highway'
}
item {
  id: 4
  name: 'mountain'
}
item {
  id: 5
  name: 'building'
}


In [ ]:
"""
15. add export_inference_graph.py and train.py from the path "Python36\Lib\site-packages\tensorflow\models\research\object_detection"
to mytraining if you want to train locally.
export_inference_graph.py is used to turn the final trained checkpoint into a frozen interface graph which cpould be used for
testing
16.Package the object detection config files on you local machine from the path:
“C:\Program Files\Python36\Lib\site-packages\tensorflow\models\research”
By running:
python setup.py sdist
cd slim
python setup.py sdist

"""

In [ ]:
#to train locally:
#go to “C:\Program Files\Python36\Lib\site-packages\tensorflow\models\research”
python object_detection/train.py \
    --logtostderr \
    --pipeline_config_path=${PATH_TO_YOUR_PIPELINE_CONFIG} \ #path to *.config I mentioned before
    --train_dir=${PATH_TO_TRAIN_DIR} #name a new folder which will contain your trained model


In [ ]:
#16.To train a model on cloud, go to :
#“C:\Program Files\Python36\Lib\site-packages\tensorflow\models\research”
#Open git bash, run the following commands, but you should change them according to your paths:
export CLOUDSDK_PYTHON="C:\Python27\python.exe" 

gcloud ml-engine jobs submit training `whoami`_object_detection_eval_`date +%s`     
--job-dir= gs://object_detection_bucket_samin /train \ #same as train_dir
--packages dist/object_detection-0.1.tar.gz,slim/dist/slim-0.1.tar.gz \    
--module-name object_detection.train  \
--region us-central1  \
 --config object_detection/samples/cloud/cloud.yml \     
--    
 --train_dir=gs:// object_detection_bucket_samin /train   \ #new folder for training jobs
--pipeline_config_path=gs://object_detection_bucket_samin/data/rfcn_resnet101_coco.config  \




In [ ]:
"""
to observe the plots and histograms of training jobs you should open the google cloud shell
from the main nav of your google cloud platform (4'th icon from right side):
And run: """
tensorboard --logdir=gs://${GCP_BUCKET_NAME} --port=8080
"""
17.After training finished you should turn your desired checkpoint of model into a frozen interface graph to use it for testing on test images. 
So, go to the path that you have your “export_inference_graph.py” and you should run this command:"""
py -3 export_inference_graph.py --input_type image_tensor --pipeline_config_path ./ssd_mobilenet_v1_coco.config --trained_checkpoint_prefix ./models/ train5_rfcn /model.ckpt-100004 --output_directory ./fine_tuned_model_5classes
#Do not forget to change the paths.



In [ ]:
"""
18.To test your images, you could use a prepared file called “object_detection_tutorial.ipynb” in the path:
“Python36\Lib\site-packages\tensorflow\models\research\object_detection”. 
You should open this file in Jupyter and follow the instructions in commands. 
You should change some paths to frozen file etc. It’s straight forward. I converted this file to python, I put it bellow"""
#object_detection_tutorial.py



# coding: utf-8

# # Object Detection Demo
# Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# # Imports

# In[1]:


import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')


# ## Env setup

# In[2]:


# This is needed to display the images.
get_ipython().run_line_magic('matplotlib', 'inline')

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")


# ## Object detection imports
# Here are the imports from the object detection module.

# In[3]:


from utils import label_map_util

from utils import visualization_utils as vis_util


# # Model preparation 

# ## Variables
# 
# Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  
# 
# By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

# In[4]:


# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90


# ## Download Model

# In[5]:


opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())


# ## Load a (frozen) Tensorflow model into memory.

# In[6]:


detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')


# ## Loading label map
# Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

# In[7]:


label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


# ## Helper code

# In[8]:


def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)


# # Detection

# In[9]:


# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


# In[10]:


with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    # Definite input and output Tensors for detection_graph
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    for image_path in TEST_IMAGE_PATHS:
      image = Image.open(image_path)
      # the array based representation of the image will be used later in order to prepare the
      # result image with boxes and labels on it.
      image_np = load_image_into_numpy_array(image)
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      # Actual detection.
      (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      # Visualization of the results of a detection.
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8)
      plt.figure(figsize=IMAGE_SIZE)
      plt.imshow(image_np)

